# Détente sur paroi centrée


On vous propose de mettre en place ...

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from aerokit.common import defaultgas
from aerokit.aero import degree     as deg  # import trigo functions with degree unit support
#from aerokit.aero import ShockWave  as sw   # import functions for shockwave computation
from aerokit.aero import Supersonic as sup  # import functions for steady supersonic flows
%matplotlib inline
#
#plt.rc('text', usetex=True)
sty_carac = { 'color': 'orange', 'linewidth': 2 }
sty_wall  = { 'color': 'black',  'linewidth': 3 }
sty_flow  = { 'color': 'green',  'linewidth': 3 }
sty_text  = { 'fontsize': 14 }

On définit tout d'abord les paramètres de ce cas. Ils sont définis comme des variables globales, que l'on peut utiliser directement dans les fonctions (comme $\gamma$ par exemple).

In [ ]:
# definition of problem parameters
gam  = 1.4 ; defaultgas.set_gamma(gam)
ymin = -1.
ymax = 1. # 
xmax = 2.
xneg = -.5
#
def init_case(m, ang):
    global M0, wang, wang0, wang1, om0, om1, M1, mu0, mu1, p1p0
    M0   = m
    wang = ang
    wang0 = 0.
    wang1 = wang
    wdev = wang1-wang0
    om0  = sup.PrandtlMeyer_Mach(M0)
    om1  = om0 - wdev
    M1   = sup.Mach_PrandtlMeyer(om1)
    p1p0 = sup.IsentropicPsratio_Mach_deflection(M1, wdev)
    mu0 = deg.asin(1./M0)
    mu1 = deg.asin(1./M1)
    
# function to plot the geometry
#
def init_fig(zoom=1):
    global ax, fig
    fig = plt.figure(figsize=(14*zoom,8*zoom), facecolor='white')
    ax  = fig.add_subplot(111) 
    #plt.axis([xneg, length])
    ax.set(aspect="equal")
    ax.axis('off')
    #ax.set_facecolor('white') # depends on backend
    ax.set_xlim(xneg, xmax) #, xlim=[xneg, 4*length], ylim=[-.1, ymax])
    ax.set_ylim(ymin, ymax) #, xlim=[xneg, 4*length], ylim=[-.1, ymax])

def plot_geom(xneg=-.5, length=xmax, ymax=ymax):
    ylow = xmax*deg.tan(wang)
    plt.fill([xneg, 0, xmax, xmax, xneg], 
             [0,    0, ylow, ymin, ymin], facecolor='lightgray', alpha=1., zorder=100)
    plt.plot([xneg, 0, xmax], 
             [0, 0, ylow], zorder=101, **sty_wall)
    plt.text(xneg+.1, -.1+xmax*deg.sin(wang)/3, r'$\Psi={:.1f}^o$'.format(wang), zorder=101, **sty_text)
    plt.text(xneg+.1, ymax-.1, r'$M_0={:.2f}$'.format(M0), zorder=101, backgroundcolor='white', **sty_text)
    plt.tight_layout()
#
# test de la fonction de tracé   
init_fig(zoom=.5); plot_geom()

## Faisceau de détente

In [ ]:
def plot_expansion(): # tracé des caractéristiques de la détente
    l1, = plt.plot([0, xmax], [0, xmax*deg.tan(wang0+mu0)], **sty_carac)
    l2, = plt.plot([0, xmax], [0, xmax*deg.tan(wang1+mu1)], **sty_carac)
    
def fill_expansion(): # tracé des caractéristiques de la détente
    poly =plt.fill([xmax, 0, xmax], [xmax*deg.tan(wang1+mu1), 0, xmax*deg.tan(wang0+mu0)], 
             facecolor=sty_carac['color'], alpha=.2)
    return poly
#
init_fig()
plot_expansion()
plot_geom()


## integration trajectoire 



In [ ]:
def flow_prop(x, y):
    oang = deg.atan2(y, x)
    if oang > wang0+mu0:
        prop = {'ang': wang0, 'C+': wang0+mu0, 'C-': wang0-mu0}
    elif oang < wang1+mu1:
        prop = {'ang': wang1, 'C+': wang1+mu1, 'C-': wang1-mu1}
    else:
        M  = sup.Mach_PMFmmu(wang0+om0-oang)
        mu = deg.asin(1./M)
        prop = {'ang': oang-mu, 'C+': oang, 'C-': oang-2*mu}
    return prop

def integ(x, y, ctype, smin, smax, npts=100):
    def step(x0, y0, ds, ang):
        return x0+ds*deg.cos(ang), y0+ds*deg.sin(ang)
    trajx = np.zeros(npts+1)
    trajy = np.zeros(npts+1)
    nforw = int(npts * smax/(smax-smin))
    nback = npts-nforw
    #print(smin, smax, nforw, nback)
    sx = x
    sy = y
    trajx[nback] = sx ; trajy[nback] = sy
    # backward
    if nback>0:
        ds = smin/nback
        for i in range(nback):
            px, py = step(sx, sy, .5*ds, flow_prop(sx, sy)[ctype])    # RK2 / predictor
            sx, sy = step(sx, sy,    ds, flow_prop(px, py)[ctype])    # RK2 / final step
            trajx[nback-i-1] = sx ; trajy[nback-i-1] = sy
    # forward
    if nforw>0:
        ds = smax/nforw
        sx = x
        sy = y
        for i in range(nforw):
            px, py = step(sx, sy, .5*ds, flow_prop(sx, sy)[ctype])    # RK2 / predictor
            sx, sy = step(sx, sy,    ds, flow_prop(px, py)[ctype])    # RK2 / final step
            trajx[nback+i+1] = sx ; trajy[nback+i+1] = sy
    return trajx, trajy
    

In [ ]:
init_fig()
fill_expansion()
#
x0 = 0. ; y0 = .2
#print(flow_prop(x0, y0), x0, y0, deg.atan2(y0,x0))
CTx, CTy = integ(x0, y0, 'ang', -.2, 3.)
plt.plot(CTx, CTy, **sty_flow)
CMx, CMy = integ(x0, y0, 'C-', -.2, .8)
CPx, CPy = integ(x0, y0, 'C+', -.2, 2.)
plt.plot(CMx, CMy, CPx, CPy, **sty_carac)
#
x0 = 0. ; y0 = .22
#print(flow_prop(x0, y0), x0, y0, deg.atan2(y0,x0))
CTx, CTy = integ(x0, y0, 'ang', 0., 3.)
plt.plot(CTx, CTy, **sty_flow)
#CMx, CMy = integ(x0, y0, 'C-', 0., .8)
#CPx, CPy = integ(x0, y0, 'C+', 0., 2.)
#plt.plot(CMx, CMy, CPx, CPy, **sty_carac)
#
plot_geom(length=2.)
#print ax.get_children()

# Backward characteristics

In [ ]:
init_fig()
fill_expansion()
#
x0 = 1.2 ; y0 = x0*deg.tan(wang1) +.2
print(flow_prop(x0, y0), x0, y0, deg.atan2(y0,x0))
CTx, CTy = integ(x0, y0, 'ang', -1.5, .8)
plt.plot(CTx, CTy, **sty_flow)
CMx, CMy = integ(x0, y0, 'C-', -2., .4)
CPx, CPy = integ(x0, y0, 'C+', -1., .4)
plt.plot(CMx, CMy, CPx, CPy, **sty_carac)
#
plot_geom(length=2.)


In [ ]:
from celluloid import Camera as cam
plt.rcParams["animation.html"] = "html5"
plt.rcParams['figure.dpi']  = 80
plt.rcParams['savefig.dpi'] = 100
#
init_fig()
camera = cam(fig)

for m in np.linspace(1.4, 4., 53):
    init_case(m, -10.)
    fill_expansion()
    x0 = 1.2 ; y0 = 0.
    CTx, CTy = integ(x0, y0, 'ang', -1.5, .8)
    plt.plot(CTx, CTy, **sty_flow)
    CMx, CMy = integ(x0, y0, 'C-', -2., .4)
    CPx, CPy = integ(x0, y0, 'C+', -1., .4)
    plt.plot(CMx, CMy, CPx, CPy, **sty_carac)
    plot_geom()
    camera.snap()

camera.animate()

In [ ]:
init_fig()
camera = cam(fig)

for a in np.linspace(-5., -25, 41):
    init_case(2., a)
    fill_expansion()
    x0 = 1.2 ; y0 = 0.
    CTx, CTy = integ(x0, y0, 'ang', -1.5, .8)
    plt.plot(CTx, CTy, **sty_flow)
    CMx, CMy = integ(x0, y0, 'C-', -2., .4)
    CPx, CPy = integ(x0, y0, 'C+', -1., .4)
    plt.plot(CMx, CMy, CPx, CPy, **sty_carac)
    plot_geom()
    camera.snap()

camera.animate()

---

<a id="ipython"></a>
## Ipython et notebook : usage

* le notebook utilise la langage de base python en version améliorée, Ipython, qui permet la complétion des noms (variables, fonctions, modules) avec la touche tabulation
* toutes les cellules peuvent être modifiées par un double-clic et sont réinterprêtées avec `shift-entrée`
* l'ensemble de la feuille peut être exécutée avec le menu `Cell/run all cells`
* **n'oubliez pas de sauvegarder régulièrement votre feuille** (bouton _enregistrer_)


In [ ]:
from IPython.core.display import HTML ; HTML(open("../custom.css", "r").read()) # notebook style